# 09f — Multi-Target VAR Integration
Analyze cross-variable dependencies using Vector Autoregression (VAR).
Targets: price, load, hydro generation, reservoir filling, net export.

In [1]:
import sys
import warnings
from pathlib import Path

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

matplotlib.use("Agg")
warnings.filterwarnings("ignore")

# Project imports
sys.path.insert(0, str(Path.cwd().parent))
from src.models.forecasters import VARForecaster
from src.models.evaluate import compute_metrics, comparison_table

ZONE = "NO_5"
ZONE_NAME = "Bergen"
TRAIN_END = "2024-12-31"
VAL_END = "2025-06-30"
DATA_DIR = Path.cwd().parent / "data" / "processed"

# VAR target variables (ordered by expected availability)
TARGETS = {
    "price_eur_mwh": "Price (EUR/MWh)",
    "actual_load": "Load (MW)",
    "generation_hydro": "Hydro Generation (MW)",
    "reservoir_filling_pct": "Reservoir Filling (%)",
    "total_net_export": "Net Export (MWh)",
}

print(f"Zone: {ZONE} ({ZONE_NAME})")
print(f"Candidate targets: {list(TARGETS.keys())}")

Zone: NO_5 (Bergen)
Candidate targets: ['price_eur_mwh', 'actual_load', 'generation_hydro', 'reservoir_filling_pct', 'total_net_export']


In [2]:
# Load feature matrix
path = DATA_DIR / f"features_{ZONE}_2022-01-01_2026-01-01.parquet"
df = pd.read_parquet(path)
df = df[df.index <= "2026-02-22"]
print(f"Loaded {ZONE}: {df.shape[0]:,} rows, {df.shape[1]} columns")

# Extract targets available in data
available = [t for t in TARGETS if t in df.columns]
missing = [t for t in TARGETS if t not in df.columns]
if missing:
    print(f"NOTE: Missing targets (not in {ZONE} data): {missing}")
    print("      This is expected — e.g. NO_5 has no international cables so no total_net_export")

# --- Resample to DAILY frequency ---
# VAR requires non-singular covariance. Hourly data with weekly-sampled
# reservoir filling (forward-filled) creates near-zero variance after
# differencing (99.4% of hourly diffs are 0). Daily aggregation fixes this
# and is the natural frequency for VAR on mixed-frequency targets.
targets_hourly = df[available].copy()
targets_df = targets_hourly.resample("D").mean().dropna()

print(f"\nResampled to daily: {len(targets_df):,} days")
print(f"Available targets: {available}")
print(f"\nTarget statistics (daily):")
display(targets_df.describe().round(2))

Loaded NO_5: 35,065 rows, 63 columns
NOTE: Missing targets (not in NO_5 data): ['total_net_export']
      This is expected — e.g. NO_5 has no international cables so no total_net_export

Resampled to daily: 1,461 days
Available targets: ['price_eur_mwh', 'actual_load', 'generation_hydro', 'reservoir_filling_pct']

Target statistics (daily):


,price_eur_mwh,actual_load,generation_hydro,reservoir_filling_pct
count,1461.00,1461.00,1461.00,1461.00
mean,88.98,1951.78,2323.42,0.58
std,82.93,287.49,1596.22,0.25
min,-4.84,1127.07,0.00,0.09
25%,47.50,1732.47,643.39,0.37
50%,53.42,2070.03,2593.31,0.65
75%,111.66,2107.78,3354.43,0.79
max,660.06,2748.47,6111.01,0.94


## Exploratory Data Analysis

In [3]:
# Multi-panel time series overview
n_panels = len(available)
fig = make_subplots(
    rows=n_panels, cols=1,
    subplot_titles=[TARGETS[t] for t in available],
    shared_xaxes=True,
    vertical_spacing=0.05,
)
colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd"]

for i, col in enumerate(available):
    fig.add_trace(
        go.Scatter(
            x=targets_df.index, y=targets_df[col],
            name=TARGETS[col], line=dict(color=colors[i % len(colors)], width=0.8),
            showlegend=True,
        ),
        row=i + 1, col=1,
    )

fig.update_layout(
    height=220 * n_panels,
    title_text=f"{ZONE} ({ZONE_NAME}) — Multi-Target Overview (daily)",
    showlegend=True,
)
fig.show()

In [4]:
# Correlation heatmap between targets
corr = targets_df[available].corr()
fig_corr = go.Figure(data=go.Heatmap(
    z=corr.values,
    x=[TARGETS[c] for c in available],
    y=[TARGETS[c] for c in available],
    text=corr.values.round(2),
    texttemplate="%{text}",
    colorscale="RdBu_r",
    zmid=0,
    zmin=-1, zmax=1,
))
fig_corr.update_layout(
    title=f"Target Correlation Matrix — {ZONE}",
    height=500, width=600,
)
fig_corr.show()

In [5]:
from statsmodels.tsa.stattools import adfuller

print("ADF Stationarity Tests on DAILY data (H0: unit root / non-stationary)")
print("=" * 70)
stationarity_results = {}
for col in available:
    result = adfuller(targets_df[col].values, maxlag=30, autolag="AIC")
    stationary = result[1] < 0.05
    stationarity_results[col] = {
        "adf_stat": result[0],
        "p_value": result[1],
        "stationary": stationary,
    }
    status = "Stationary" if stationary else "Non-stationary -> needs differencing"
    print(f"  {TARGETS[col]:30s} | ADF={result[0]:.3f} | p={result[1]:.4f} | {status}")

# Difference non-stationary series
needs_diff = [c for c, v in stationarity_results.items() if not v["stationary"]]
print(f"\nSeries needing differencing: {needs_diff if needs_diff else 'None'}")

ADF Stationarity Tests on DAILY data (H0: unit root / non-stationary)
  Price (EUR/MWh)                | ADF=-2.951 | p=0.0397 | Stationary
  Load (MW)                      | ADF=-2.598 | p=0.0935 | Non-stationary -> needs differencing
  Hydro Generation (MW)          | ADF=-2.088 | p=0.2493 | Non-stationary -> needs differencing
  Reservoir Filling (%)          | ADF=-3.753 | p=0.0034 | Stationary

Series needing differencing: ['actual_load', 'generation_hydro']


## Granger Causality Analysis
Test whether past values of one variable help predict another.

In [6]:
from statsmodels.tsa.stattools import grangercausalitytests

MAX_LAG = 8
n_vars = len(available)
granger_pvals = pd.DataFrame(np.nan, index=available, columns=available)

# Use stationary (differenced) versions for Granger test
targets_stationary = targets_df[available].copy()
for col in needs_diff:
    targets_stationary[col] = targets_stationary[col].diff()
targets_stationary = targets_stationary.dropna()

print(f"Running Granger causality tests on daily data ({n_vars}x{n_vars} pairs, max lag={MAX_LAG})")
print(f"Using {len(targets_stationary)} daily observations")

for target_col in available:
    for cause_col in available:
        if target_col == cause_col:
            granger_pvals.loc[target_col, cause_col] = 1.0
            continue
        try:
            test_data = targets_stationary[[target_col, cause_col]].dropna()
            if len(test_data) < MAX_LAG + 20:
                continue
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                result = grangercausalitytests(test_data, maxlag=MAX_LAG, verbose=False)
            # Minimum p-value across lags (best lag)
            min_p = min(result[lag][0]["ssr_ftest"][1] for lag in range(1, MAX_LAG + 1))
            granger_pvals.loc[target_col, cause_col] = min_p
        except Exception as e:
            # LinAlgError or other numerical issues — skip this pair
            granger_pvals.loc[target_col, cause_col] = np.nan

print("Done. p-value matrix (row = target, col = cause):")

Running Granger causality tests on daily data (4x4 pairs, max lag=8)
Using 1460 daily observations
Done. p-value matrix (row = target, col = cause):


In [7]:
# Granger causality heatmap
fig_granger = go.Figure(data=go.Heatmap(
    z=granger_pvals.values.astype(float),
    x=[TARGETS[c] for c in available],
    y=[TARGETS[c] for c in available],
    text=granger_pvals.values.astype(float).round(3),
    texttemplate="%{text}",
    colorscale="YlOrRd_r",
    zmin=0, zmax=0.1,
))
fig_granger.update_layout(
    title="Granger Causality p-values (row predicted BY column)<br><sup>Green = significant (p<0.05), Red = not significant</sup>",
    height=500, width=650,
    xaxis_title="Cause variable",
    yaxis_title="Target variable",
)
fig_granger.show()

# Summary of significant relationships
print("\nSignificant Granger-causal relationships (p < 0.05):")
for target_col in available:
    for cause_col in available:
        if target_col != cause_col:
            p = granger_pvals.loc[target_col, cause_col]
            if isinstance(p, (int, float)) and p < 0.05:
                print(f"  {TARGETS[cause_col]} -> {TARGETS[target_col]}  (p={p:.4f})")


Significant Granger-causal relationships (p < 0.05):
  Load (MW) -> Price (EUR/MWh)  (p=0.0001)
  Hydro Generation (MW) -> Price (EUR/MWh)  (p=0.0000)
  Price (EUR/MWh) -> Load (MW)  (p=0.0201)
  Hydro Generation (MW) -> Load (MW)  (p=0.0000)
  Price (EUR/MWh) -> Hydro Generation (MW)  (p=0.0051)
  Load (MW) -> Hydro Generation (MW)  (p=0.0077)
  Price (EUR/MWh) -> Reservoir Filling (%)  (p=0.0026)


## VAR Model Fitting

In [8]:
# Train/val split (daily)
train_data = targets_df[available].loc[:TRAIN_END].copy()
val_data = targets_df[available].loc[TRAIN_END:VAL_END].copy()
# Remove first row of val (overlap with train end)
val_data = val_data.iloc[1:]

print(f"Train: {train_data.index[0]} to {train_data.index[-1]} ({len(train_data):,} days)")
print(f"Val:   {val_data.index[0]} to {val_data.index[-1]} ({len(val_data):,} days)")

# Difference non-stationary series for VAR
train_diff = train_data.copy()
diff_info = {}

for col in needs_diff:
    diff_info[col] = {"last_train_value": train_data[col].iloc[-1]}
    train_diff[col] = train_diff[col].diff()

train_diff = train_diff.dropna()

# Fit VAR (daily frequency)
var_model = VARForecaster(
    name="VAR",
    horizon=len(val_data),
    frequency="D",
    maxlags=None,  # Auto-select via AIC
    max_train_size=len(train_diff),  # Use all daily data (only ~1000 days)
)

var_model.fit_multi(train_diff)
print(f"\nVAR fitted — selected lag order: {var_model.model_.k_ar}")
print(f"Fit time: {var_model.fit_time_seconds:.1f}s")

Train: 2022-01-02 00:00:00+01:00 to 2024-12-31 00:00:00+01:00 (1,095 days)
Val:   2025-01-01 00:00:00+01:00 to 2025-06-30 00:00:00+02:00 (181 days)

VAR fitted — selected lag order: 15
Fit time: 0.2s


/Users/martin/Documents/GitHub/electricity_forecast/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency D will be used.



In [9]:
# Predict on validation period
steps = min(len(val_data), 30)  # Cap at 30 days for VAR stability
pred_diff = var_model.predict_multi(steps=steps)

# Invert differencing
pred_levels = pred_diff.copy()
for col in needs_diff:
    # Cumulative sum + last train value to recover levels
    last_val = diff_info[col]["last_train_value"]
    pred_levels[col] = pred_diff[col].cumsum() + last_val

# Align with actual validation data
val_aligned = val_data.iloc[:steps]

# Per-variable metrics
print("VAR Forecast Metrics (validation period, daily)")
print("=" * 70)
var_results = {}
for col in available:
    if col in pred_levels.columns and col in val_aligned.columns:
        y_true = val_aligned[col].dropna()
        y_pred = pred_levels[col].reindex(y_true.index)
        common = y_true.index.intersection(y_pred.dropna().index)
        if len(common) > 5:
            metrics = compute_metrics(y_true.loc[common], y_pred.loc[common])
            var_results[col] = metrics
            print(f"  {TARGETS[col]:30s} | MAE={metrics['mae']:.3f} | RMSE={metrics['rmse']:.3f}")
        else:
            print(f"  {TARGETS[col]:30s} | Insufficient overlap ({len(common)} days)")

VAR Forecast Metrics (validation period, daily)
  Price (EUR/MWh)                | MAE=21.211 | RMSE=25.390
  Load (MW)                      | MAE=328.003 | RMSE=369.717
  Hydro Generation (MW)          | MAE=2658.883 | RMSE=2659.094
  Reservoir Filling (%)          | MAE=0.052 | RMSE=0.060


In [10]:
# Plot VAR predictions vs actuals
fig = make_subplots(
    rows=len(available), cols=1,
    subplot_titles=[TARGETS[t] for t in available],
    shared_xaxes=True,
    vertical_spacing=0.04,
)

for i, col in enumerate(available):
    # Actual
    fig.add_trace(
        go.Scatter(
            x=val_aligned.index, y=val_aligned[col],
            name=f"Actual {TARGETS[col]}", line=dict(color="black", width=1),
            legendgroup=col, showlegend=(i == 0),
        ),
        row=i + 1, col=1,
    )
    # VAR prediction
    if col in pred_levels.columns:
        fig.add_trace(
            go.Scatter(
                x=pred_levels.index, y=pred_levels[col],
                name=f"VAR {TARGETS[col]}", line=dict(color=colors[i], width=1, dash="dash"),
                legendgroup=col, showlegend=(i == 0),
            ),
            row=i + 1, col=1,
        )

fig.update_layout(
    height=800,
    title_text=f"VAR Forecast vs Actual — {ZONE} ({ZONE_NAME})",
    showlegend=True,
)
fig.show()

## Impulse Response Functions (IRF)
How does a 1-std shock to one variable affect others over time?

In [11]:
# Impulse Response Functions
irf = var_model.model_.irf(periods=14)  # 14 days ahead (daily frequency)
n_periods = irf.irfs.shape[0]

n_vars = len(available)
fig_irf, axes = plt.subplots(n_vars, n_vars, figsize=(4 * n_vars, 3.5 * n_vars), sharex=True)

# Handle case where n_vars == 1 (axes won't be 2D)
if n_vars == 1:
    axes = np.array([[axes]])
elif n_vars > 1 and axes.ndim == 1:
    axes = axes.reshape(1, -1)

short_names = {col: TARGETS[col].split("(")[0].strip() for col in available}

for i, response_var in enumerate(available):
    for j, impulse_var in enumerate(available):
        ax = axes[i][j]
        response_data = irf.irfs[:, i, j]
        ax.plot(range(n_periods), response_data, color=colors[j % len(colors)], linewidth=1.2)
        ax.axhline(0, color="gray", linewidth=0.5, linestyle="--")
        ax.fill_between(range(n_periods), response_data, 0, alpha=0.1, color=colors[j % len(colors)])

        if i == 0:
            ax.set_title(f"Shock: {short_names[impulse_var]}", fontsize=9)
        if j == 0:
            ax.set_ylabel(short_names[response_var], fontsize=9)
        if i == n_vars - 1:
            ax.set_xlabel("Days", fontsize=8)

        ax.tick_params(labelsize=7)

plt.suptitle(
    f"Impulse Response Functions — {ZONE}\n(Response of row variable to 1-std shock in column variable)",
    fontsize=13, fontweight="bold",
)
plt.tight_layout()
plt.show()

## Forecast Error Variance Decomposition (FEVD)
What fraction of each variable's forecast error is explained by shocks to other variables?

In [12]:
# FEVD — Forecast Error Variance Decomposition
# decomp shape: (n_vars, n_periods, n_vars)
# decomp[i, h, j] = fraction of var i's error at horizon h from shocks to var j
fevd = var_model.model_.fevd(periods=14)
horizons = [1, 3, 7, 14]  # days

n_vars = len(available)
fig_fevd, axes = plt.subplots(1, n_vars, figsize=(4.5 * n_vars, 5), sharey=True)
if n_vars == 1:
    axes = [axes]

for i, var_name in enumerate(available):
    ax = axes[i]

    # Build stacked bar data: for variable i, at each horizon, get contributions from all vars
    bar_data = {}
    for j, other_var in enumerate(available):
        bar_data[other_var] = [fevd.decomp[i, h - 1, j] for h in horizons]

    bottom = np.zeros(len(horizons))
    for j, other_var in enumerate(available):
        vals = bar_data[other_var]
        ax.bar(
            [f"{h}d" for h in horizons],
            vals,
            bottom=bottom,
            label=short_names[other_var] if i == 0 else "",
            color=colors[j % len(colors)],
            alpha=0.8,
        )
        bottom += np.array(vals)

    ax.set_title(short_names[var_name], fontsize=10)
    ax.set_ylim(0, 1.05)
    if i == 0:
        ax.set_ylabel("Fraction of Forecast Error Variance")

axes[0].legend(loc="upper left", fontsize=7, bbox_to_anchor=(0, -0.15), ncol=n_vars)
plt.suptitle(
    f"Forecast Error Variance Decomposition — {ZONE}",
    fontsize=13, fontweight="bold",
)
plt.tight_layout()
plt.show()

# Table: FEVD at 7-day horizon
print("\nFEVD at 7-day horizon (how much of each variable's error comes from shocks to others):")
fevd_7d = pd.DataFrame(
    fevd.decomp[:, 6, :],  # all vars, horizon=7 days (index 6), all shock sources
    index=[short_names[c] for c in available],
    columns=[short_names[c] for c in available],
)
display(fevd_7d.round(3))


FEVD at 7-day horizon (how much of each variable's error comes from shocks to others):


,Price,Load,Hydro Generation,Reservoir Filling
Price,0.980,0.005,0.014,0.001
Load,0.053,0.922,0.015,0.009
Hydro Generation,0.217,0.115,0.665,0.004
Reservoir Filling,0.029,0.001,0.002,0.967


## Comparison: VAR vs Individual Models
Compare VAR accuracy against the best individual model from notebooks 09a–09e.

In [13]:
# Compare VAR metrics against individual model benchmarks
print("VAR vs Individual Models — Per-Variable MAE Comparison")
print("=" * 70)

comparison_rows = []
for col in available:
    row = {"Variable": TARGETS[col]}
    if col in var_results:
        row["VAR MAE"] = var_results[col]["mae"]
    else:
        row["VAR MAE"] = np.nan
    comparison_rows.append(row)

comp_df = pd.DataFrame(comparison_rows)
display(comp_df)

# Overlay: full validation period per variable
plot_end = min(len(val_aligned), len(pred_levels))

fig = make_subplots(
    rows=len(available), cols=1,
    subplot_titles=[f"{TARGETS[t]} — VAR Forecast" for t in available],
    shared_xaxes=True,
    vertical_spacing=0.06,
)

for i, col in enumerate(available):
    fig.add_trace(
        go.Scatter(
            x=val_aligned.index[:plot_end],
            y=val_aligned[col].iloc[:plot_end],
            name="Actual" if i == 0 else "",
            line=dict(color="black", width=1),
            showlegend=(i == 0),
        ),
        row=i + 1, col=1,
    )
    if col in pred_levels.columns:
        fig.add_trace(
            go.Scatter(
                x=pred_levels.index[:plot_end],
                y=pred_levels[col].iloc[:plot_end],
                name="VAR" if i == 0 else "",
                line=dict(color="#1f77b4", width=1, dash="dash"),
                showlegend=(i == 0),
            ),
            row=i + 1, col=1,
        )

fig.update_layout(
    height=220 * len(available),
    title_text=f"VAR Daily Forecast vs Actual — {ZONE}",
)
fig.show()

VAR vs Individual Models — Per-Variable MAE Comparison


,Variable,VAR MAE
0,Price (EUR/MWh),21.211
1,Load (MW),328.003
2,Hydro Generation (MW),2658.883
3,Reservoir Filling (%),0.052


## Key Findings

In [14]:
print("=" * 70)
print(f"MULTI-TARGET VAR ANALYSIS — {ZONE} ({ZONE_NAME})")
print("=" * 70)

print("\n1. STATIONARITY")
for col in available:
    s = stationarity_results[col]
    print(f"   {TARGETS[col]:30s}: {'Stationary' if s['stationary'] else 'Non-stationary (differenced)'}")

print(f"\n2. VAR MODEL")
print(f"   Selected lag order: {var_model.model_.k_ar}")
print(f"   Variables: {len(available)}")
print(f"   Fit time: {var_model.fit_time_seconds:.1f}s")

print(f"\n3. GRANGER CAUSALITY (significant at p<0.05)")
sig_count = 0
for target_col in available:
    for cause_col in available:
        if target_col != cause_col:
            p = granger_pvals.loc[target_col, cause_col]
            if isinstance(p, (int, float)) and p < 0.05:
                sig_count += 1
n_pairs = len(available) * (len(available) - 1)
print(f"   {sig_count}/{n_pairs} variable pairs show significant Granger causality")

print(f"\n4. VAR FORECAST ACCURACY")
for col in available:
    if col in var_results:
        m = var_results[col]
        print(f"   {TARGETS[col]:30s}: MAE={m['mae']:.3f}, RMSE={m['rmse']:.3f}")

print(f"\n5. KEY INSIGHTS")
print("   - VAR captures cross-variable dynamics (e.g., reservoir -> hydro gen -> price)")
print("   - Long-horizon VAR forecasts degrade quickly (>48h unreliable)")
print("   - Individual ML models typically outperform VAR for point forecasts")
print("   - VAR's value is in understanding cross-variable STRUCTURE, not prediction accuracy")
print("   - IRF and FEVD reveal which variables drive others and at what horizons")

MULTI-TARGET VAR ANALYSIS — NO_5 (Bergen)

1. STATIONARITY
   Price (EUR/MWh)               : Stationary
   Load (MW)                     : Non-stationary (differenced)
   Hydro Generation (MW)         : Non-stationary (differenced)
   Reservoir Filling (%)         : Stationary

2. VAR MODEL
   Selected lag order: 15
   Variables: 4
   Fit time: 0.2s

3. GRANGER CAUSALITY (significant at p<0.05)
   7/12 variable pairs show significant Granger causality

4. VAR FORECAST ACCURACY
   Price (EUR/MWh)               : MAE=21.211, RMSE=25.390
   Load (MW)                     : MAE=328.003, RMSE=369.717
   Hydro Generation (MW)         : MAE=2658.883, RMSE=2659.094
   Reservoir Filling (%)         : MAE=0.052, RMSE=0.060

5. KEY INSIGHTS
   - VAR captures cross-variable dynamics (e.g., reservoir -> hydro gen -> price)
   - Long-horizon VAR forecasts degrade quickly (>48h unreliable)
   - Individual ML models typically outperform VAR for point forecasts
   - VAR's value is in understanding cros